In [1]:
%cd /global/u2/z/zeyuanhu/public_codes/ClimSim/
from climsim_utils.data_utils import *

/global/u2/z/zeyuanhu/public_codes/ClimSim


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]
2024-07-23 03:36:32.088392: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-23 03:36:32.088422: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-23 03:36:32.090115: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-23 03:36:32.098674: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in perform

In [2]:
import torch
import numpy as np
import torch.optim as optim
import torch.nn as nn
import modulus

# Create a wrapper model to include normalization and de-normalization inside model's forward method

We define below a new class "NewModel" that takes the trained U-Net model and include all the preprocessing and post-processing steps inside the forward method.

In [4]:
%cd /global/u2/z/zeyuanhu/public_codes/ClimSim/online_testing/baseline_models/Unet_v4/training
from climsim_unet import ClimsimUnet
import climsim_unet as climsim_unet

/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


/global/u2/z/zeyuanhu/public_codes/ClimSim/online_testing/baseline_models/Unet_v4/training


In [5]:
class NewModel(nn.Module):
    def __init__(self, original_model, input_sub, input_div, out_scale, lbd_qc, lbd_qi):
        super(NewModel, self).__init__()
        self.original_model = original_model
        self.input_sub = torch.tensor(input_sub, dtype=torch.float32)
        self.input_div = torch.tensor(input_div, dtype=torch.float32)
        self.out_scale = torch.tensor(out_scale, dtype=torch.float32)
        self.lbd_qc = torch.tensor(lbd_qc, dtype=torch.float32)
        self.lbd_qi = torch.tensor(lbd_qi, dtype=torch.float32)

    def preprocessing(self, x):
        
        #do input normalization
        x[:,120:180] = 1 - torch.exp(-x[:,120:180] * self.lbd_qc)
        x[:,180:240] = 1 - torch.exp(-x[:,180:240] * self.lbd_qi)
        x= (x - self.input_sub) / self.input_div
        x = torch.where(torch.isnan(x), torch.tensor(0.0, device=x.device), x)
        x = torch.where(torch.isinf(x), torch.tensor(0.0, device=x.device), x)
        
        #prune top 15 levels in qn input
        x[:,120:120+15] = 0
        x[:,180:180+15] = 0
        #clip rh input
        x[:, 60:120] = torch.clamp(x[:, 60:120], 0, 1.2)
        return x

    def postprocessing(self, x):
        x[:,60:75] = 0
        x[:,120:120+28] = 0
        x[:,180:195] = 0
        x[:,240:255] = 0
        x[:,300:315] = 0
        x = x/self.out_scale
        return x

    def forward(self, x):
        x = self.preprocessing(x)
        x = self.original_model(x)
        x = self.postprocessing(x)    
        return x

In [5]:
def save_wrapper(casename):
    # casename = 'v5_noclassifier_huber_1y_noaggressive'
    f_torch_model = f'/global/homes/z/zeyuanhu/scratch/hugging/E3SM-MMF_ne4/saved_models/{casename}/model.mdlus'
    f_inp_sub     = f'/global/homes/z/zeyuanhu/scratch/hugging/E3SM-MMF_ne4/saved_models/{casename}/inp_sub.txt'
    f_inp_div     = f'/global/homes/z/zeyuanhu/scratch/hugging/E3SM-MMF_ne4/saved_models/{casename}/inp_div.txt'
    f_out_scale   = f'/global/homes/z/zeyuanhu/scratch/hugging/E3SM-MMF_ne4/saved_models/{casename}/out_scale.txt'
    f_qc_lbd = '/global/u2/z/zeyuanhu/public_codes/ClimSim/preprocessing/normalizations/inputs/qc_exp_lambda_large.txt'
    f_qi_lbd = '/global/u2/z/zeyuanhu/public_codes/ClimSim/preprocessing/normalizations/inputs/qi_exp_lambda_large.txt'
    lbd_qc = np.loadtxt(f_qc_lbd, delimiter=',')
    lbd_qi = np.loadtxt(f_qi_lbd, delimiter=',')
    input_sub = np.loadtxt(f_inp_sub, delimiter=',')
    input_div = np.loadtxt(f_inp_div, delimiter=',')
    out_scale = np.loadtxt(f_out_scale, delimiter=',')
    model_inf = modulus.Module.from_checkpoint(f_torch_model).to('cpu')

    new_model = NewModel(model_inf, input_sub, input_div, out_scale, lbd_qc, lbd_qi)

    NewModel.device = "cpu"
    device = torch.device("cpu")
    scripted_model = torch.jit.script(new_model)
    scripted_model = scripted_model.eval()
    save_file_torch = os.path.join('/global/homes/z/zeyuanhu/scratch/hugging/E3SM-MMF_ne4/saved_models_wrapper_tmp/', f'{casename}.pt')
    scripted_model.save(save_file_torch)
    return None

In [10]:
save_wrapper('v4plus_unet_nonaggressive_cliprh_huber_rop2_r3')

In [11]:
save_wrapper('v4plus_unet_nonaggressive_cliprh_huber')

In [12]:
save_wrapper('v4plus_unet_nonaggressive_cliprh_mae')